## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list


In [ ]:
import importlib
import Python_scripts.Feature_functions.accel_outliers
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.accel_outliers)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.accel_outliers import (
    _mad, accel_outlier_mask, _get_trial_meta, accel_outliers_for_trial, batch_accel_outliers
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


## Fetch id list

In [ ]:
task_name = "AllTask"
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=None, #['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly'],
    dose_mult=2,
    genotype="white",
    bad_ids=bad_id,
    csv_prefix="dlc_table",   # -> dlc_table_saline.csv, dlc_table_ghrelin.csv, ...
    min_trial_length=None     # or 600
)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"exc_id: {exc_id}\n")
print(f"inh_id: {inh_id}\n")


## Calculate acceleration

### Batch call with different input parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
bodypart = 'Midback'          # or 'Midback'
time_limit = None          # e.g., 120 for first 2 minutes; None for full trial
smooth = True              # IMPORTANT: set True, otherwise 'window' has no effect
window_size = 25          # 11,15,19,23,27
mad_thresh = np.arange(0.5, 4.6, 0.5)

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    "Ghrelin": ghrelin_id,
    # "Inhibitory": inh_id,
    # "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_{bodypart}_acc_out_madThresh_sweep.pdf"
with PdfPages(outfile) as pdf:
    for mad_size in mad_thresh:
        print(f"[INFO] Analyzing mad_size = {mad_size}")

        # Compute all groups for this window size
        frames = []
        for label, ids in group_specs.items():
            df = batch_accel_outliers(
                conn, ids,
                bodypart=bodypart,
                time_limit=time_limit,
                smooth=smooth,
                window=window_size,
                mad_thresh=mad_size
            )
            # Keep what we need; add group label
            df = df[['trial_id', 'rate_per_min']].copy().dropna()
            df['group'] = label
            frames.append(df)

        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue

        df_acc = pd.concat(frames, ignore_index=True)
        print(df_acc.head())

        # Plot
        order = list(group_specs.keys())
        fig, ax = plot_groupwise_bar(
            df_acc,
            y='rate_per_min',
            ylabel='# of acc outlier',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ranksums", "ttest", "anova")  # if supported
        )
        ax.set_title(f"{task_name} | {bodypart} | MAD={mad_size}", pad=20)

        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")


### Single call with fixed hyper parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Configure once ---
bodypart = 'Midback'
time_limit = None
smooth = True
window = 25  # smoothing window for compute_motion_features (if smooth=True)
mad_thresh = 3

# Put every group we *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,    # lists/arrays of trial IDs, defined earlier
    "Ghrelin": ghrelin_id,
    # "Inhibitory": inh_id,
    # "Excitatory": exc_id,
}

# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

# Compute all groups
frames = []
for label, ids in group_specs.items():
    df = batch_accel_outliers(
        conn, ids,
        bodypart=bodypart,
        time_limit=time_limit,
        smooth=smooth,
        window=window,
        mad_thresh=mad_thresh
    )
    # Keep just what we need for plotting; add group label
    df = df[['trial_id', 'rate_per_min']].copy().dropna()
    df['group'] = label
    frames.append(df)

if not frames:
    raise ValueError("[WARN] No groups provided—nothing to plot.")

df_accel = pd.concat(frames, ignore_index=True)

# Plot (uses your existing plotter)
order = list(group_specs.keys())  # consistent ordering across pages
fig, ax = plot_groupwise_bar(
    df_accel,
    y='rate_per_min',
    ylabel='Acceleration outlier rate (spikes/min)',
    plot_type='bar',
    show_points=False,
    order=order,
    show_stats=True,
    tests_to_show=("ranksums", "ttest", "anova")  # optional, if your function supports it
)

ax.set_title(f"{task_name} | {bodypart} | window={window}", pad=20)

fig.savefig(f"White_10X_{task_name}_acc_out.pdf", dpi=300, bbox_inches='tight')
